<a href="https://colab.research.google.com/github/KamonohashiPerry/MachineLearning/blob/master/Causal_Inference/Python_Causal_Inference_Chap7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ベイジアンネットワーク
+ DAGを前提としている
+ グラフ表現により変数間の関係性を示すための手法
+ 構造方程式は使わないで、条件付き確率表を用いる。

### 関連用語
+ スケルトン
 + ノードとノードの繋がりだけを表す（やじるしなし）
+ PDAG
 + 一部は有向で一部が無向のグラフ

### ネットワークの当てはまりの良さ
+ AIC
+ BIC
+ BDeu
+ MDL符号
+ BDe
+ BDeu
+ K2

In [51]:
# 乱数のシードを設定
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)

from numpy.random import *
import pandas as pd

データの作成

In [52]:
# データ数
num_data = 10

# x1:0か1の値をnum_data個生成、0の確率は0.6、1の確率は0.4
x1 = np.random.choice([0, 1], num_data, p=[0.6, 0.4])

# x2:0か1の値をnum_data個生成、0の確率は0.4、1の確率は0.6
x2 = np.random.choice([0, 1], num_data, p=[0.4, 0.6])

# 2変数で表にする
df = pd.DataFrame({'x1':x1,
                    'x2':x2})
df.head()

,x1,x2
0,0,0
1,1,1
2,0,1
3,1,1
4,1,0


In [53]:
# 変数x3:0か1の値をnum_data個生成する
# (x1, x2) = (0, 0)のとき、0の確率は0.2
# (x1, x2) = (1, 0)のとき、0の確率は0.3
# (x1, x2) = (0, 1)のとき、0の確率は0.4
# (x1, x2) = (1, 1)のとき、0の確率は0.1

x3 = []
for i in range(num_data):
  if x1[i] == 0 and x2[i] == 0:
    x3_value = np.random.choice([0, 1], 1, p=[0.2, 0.8])
    x3.append(x3_value[0])
  elif x1[i] == 0 and x2[i] == 1:
    x3_value = np.random.choice([0, 1], 1, p=[0.3, 0.7])
    x3.append(x3_value[0])
  elif x1[i] == 1 and x2[i] == 0:
    x3_value = np.random.choice([0, 1], 1, p=[0.4, 0.6])
    x3.append(x3_value[0])
  elif x1[i] == 1 and x2[i] == 1:
    x3_value = np.random.choice([0, 1], 1, p=[0.1, 0.9])
    x3.append(x3_value[0])

df['x3'] = x3

df

,x1,x2,x3
0,0,0,1
1,1,1,1
2,0,1,0
3,1,1,1
4,1,0,1
5,0,1,1
6,0,1,1
7,1,0,1
8,1,1,1
9,1,1,1


In [54]:
 !pip install pgmpy==0.1.9

In [55]:
# 正解のDAGを与える
from pgmpy.models import BayesianModel

model = BayesianModel([('x1', 'x3'), ('x2', 'x3')])

In [56]:
# 各データパターンの個数を表示する
from pgmpy.estimators import ParameterEstimator
pe = ParameterEstimator(model, df)
print("\n", pe.state_counts('x1'))
print("\n", pe.state_counts('x2'))
print("\n", pe.state_counts('x3'))


    x1
0   4
1   6

    x2
0   3
1   7

 x1    0         1     
x2    0    1    0    1
x3                    
0   0.0  1.0  0.0  0.0
1   1.0  2.0  2.0  4.0


In [57]:
# CPT（条件付き確率表）を推定する
from pgmpy.estimators import BayesianEstimator

estimator = BayesianEstimator(model, df)

cpd_x1 = estimator.estimate_cpd('x1', prior_type='dirichlet', pseudo_counts=[[0], [0]])
cpd_x2 = estimator.estimate_cpd('x2', prior_type='dirichlet', pseudo_counts=[[0], [0]])
cpd_x3 = estimator.estimate_cpd('x3', prior_type='dirichlet', pseudo_counts=[[0,0,0,0], [0,0,0,0]])

print(cpd_x1)
print(cpd_x2)
print(cpd_x3)

+-------+-----+
| x1(0) | 0.4 |
+-------+-----+
| x1(1) | 0.6 |
+-------+-----+
+-------+-----+
| x2(0) | 0.3 |
+-------+-----+
| x2(1) | 0.7 |
+-------+-----+
+-------+-------+--------------------+-------+-------+
| x1    | x1(0) | x1(0)              | x1(1) | x1(1) |
+-------+-------+--------------------+-------+-------+
| x2    | x2(0) | x2(1)              | x2(0) | x2(1) |
+-------+-------+--------------------+-------+-------+
| x3(0) | 0.0   | 0.3333333333333333 | 0.0   | 0.0   |
+-------+-------+--------------------+-------+-------+
| x3(1) | 1.0   | 0.6666666666666666 | 1.0   | 1.0   |
+-------+-------+--------------------+-------+-------+


In [58]:
# BICを求める
from pgmpy.estimators import BicScore
bic = BicScore(df)
print(bic.score(model))

-21.65605747450808


In [59]:
# 正解ではないDAGを与える
from pgmpy.models import BayesianModel
model = BayesianModel([('x2', 'x1'), ('x2', 'x3')])
bic = BicScore(df)
print(bic.score(model))

-21.425819218840655


## 変数間の独立性の検定

In [60]:
# 乱数のシードを設定
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)

from numpy.random import *
import pandas as pd

In [61]:
# データ数
num_data = 100

# x1:0か1の値をnum_data個生成、0の確率は0.6、1の確率は0.4
x1 = np.random.choice([0, 1], num_data, p=[0.6, 0.4])

# x2:0か1の値をnum_data個生成、0の確率は0.4、1の確率は0.6
x2 = np.random.choice([0, 1], num_data, p=[0.4, 0.6])

# x2はx1と因果関係にあるとする
x2 = x2*x1

# 2変数で表にする
df = pd.DataFrame({'x1':x1,
                     'x2':x2})

df.head()

,x1,x2
0,0,0
1,1,1
2,0,0
3,1,1
4,1,1


In [62]:
# 各カウント
print(((df['x1'] == 0) & (df['x2'] == 0)).sum())
print(((df['x1'] == 1) & (df['x2'] == 0)).sum())
print(((df['x1'] == 0) & (df['x2'] == 1)).sum())
print(((df['x1'] == 1) & (df['x2'] == 1)).sum())

58
9
0
33


### 条件付きの独立性のカイ二乗検定

In [63]:
# データ数
num_data = 100

# x1:0か1の値をnum_data個生成、0の確率は0.6、1の確率は0.4
x1 = np.random.choice([0, 1], num_data, p=[0.6, 0.4])

# x2:0か1の値をnum_data個生成、0の確率は0.4、1の確率は0.6
x2 = np.random.choice([0, 1], num_data, p=[0.4, 0.6])

# 2変数で表にする
df2 = pd.DataFrame({'x1':x1,
                     'x2':x2})

# 各カウント
print(((df['x1'] == 0) & (df['x2'] == 0)).sum())
print(((df['x1'] == 1) & (df['x2'] == 0)).sum())
print(((df['x1'] == 0) & (df['x2'] == 1)).sum())
print(((df['x1'] == 1) & (df['x2'] == 1)).sum())

58
9
0
33


In [64]:
from pgmpy.estimators import ConstraintBasedEstimator

est = ConstraintBasedEstimator(df2)
print(est.test_conditional_independence('x1', 'x2', method='chi_square', tol=0.05))

# 最初の例の場合
est = ConstraintBasedEstimator(df)
print(est.test_conditional_independence('x1', 'x2', method='chi_square', tol=0.05))

True
False


## PCアルゴリズムによるベイジアンネットワーク探索の実装

In [65]:
# 乱数のシードの設定
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)

# 標準正規分布の生成用
from numpy.random import *

# グラフの描画用
import matplotlib.pyplot as plt

# SciPy 平均0、分散1に正規化する関数
import scipy.stats

# シグモイド関数
from scipy.special import expit

import pandas as pd

In [66]:
# データ数
num_data = 2000

# 部下育成への熱心さ
x = np.random.uniform(low=-1, high=1, size=num_data)

# 上司が研修に参加したかどうか
e_z = randn(num_data)
z_prob = expit(-0.5*x + 5*e_z)
Z = np.array([])

for i in range(num_data):
  Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0]
  Z = np.append(Z, Z_i)

# 介入効果の非線形性
t = np.zeros(num_data)
for i in range(num_data):
  if x[i] < 0:
    t[i] = 0.5
  elif x[i] >= 0 and x[i] < 0.5:
    t[i] = 0.7
  elif x[i] >= 0.5:
    t[i] = 1.0

e_y = randn(num_data)
Y = 2.0 + t*Z + 0.3*x + 0.1*e_y

# 部下当人のチームメンバーへの満足度
Y2 = np.random.choice([1.0, 2.0, 3.0, 4.0, 5.0],
                        num_data, p=[0.1, 0.2, 0.3, 0.2, 0.2])

# 部下当人の仕事への満足度
e_y3 = randn(num_data)
Y3 = 3*Y + Y2 + e_y3

# 部下当人の仕事のパフォーマンス
e_y4 = randn(num_data)
Y4 = 3*Y3 + 5 + 2*e_y4

df = pd.DataFrame({'x':x,
                    'Z':Z,
                    't':t,
                    'Y':Y,
                    'Y2':Y2,
                    'Y3':Y3,
                    'Y4':Y4})
df.head()

,x,Z,t,Y,Y2,Y3,Y4
0,-0.616961,1.0,0.5,2.286924,2.0,8.732544,30.326507
1,0.244218,1.0,0.7,2.864636,3.0,10.743959,37.149014
2,-0.124545,0.0,0.5,2.198515,3.0,10.569163,38.481185
3,0.570717,1.0,1.0,3.230572,3.0,12.312526,43.709229
4,0.559952,1.0,1.0,3.459267,5.0,15.418739,49.833938


### データの離散化

In [67]:
# データを区切る
df_bin = df.copy()
del df_bin['t']

# 部下育成への熱心さ
df_bin['x'], x_bins = pd.cut(df['x'], 5, labels=[1, 2, 3, 4, 5], retbins=True)

# 上司が研修に参加したかどうか
df_bin['Z'], z_bins = pd.cut(df['Z'], 2, labels=[0,1], retbins=True)

# 部下当人の面談の満足度
df_bin['Y'], y_bins = pd.cut(df['Y'], 5, labels=[1,2,3,4,5], retbins=True)

# 部下当人のチームメンバーへの満足度
df_bin['Y2'], y2_bins = pd.cut(df['Y2'], 5, labels=[1,2,3,4,5], retbins=True)

# 部下当人の仕事への満足度
df_bin['Y3'], y3_bins = pd.cut(df['Y3'], 5, labels=[1,2,3,4,5], retbins=True)

# 部下当人の仕事のパフォーマンス
df_bin['Y4'], y4_bins = pd.cut(df['Y4'], 5, labels=[1,2,3,4,5], retbins=True)

df_bin.head()

,x,Z,Y,Y2,Y3,Y4
0,1,1,2,2,2,2
1,4,1,4,3,3,3
2,3,0,2,3,3,3
3,4,1,5,3,4,4
4,4,1,5,5,5,4


### PCアルゴリズムによる因果探索：0次の独立性

In [68]:
from pgmpy.estimators import ConstraintBasedEstimator

est = ConstraintBasedEstimator(df_bin)

# 0次の独立性の検定
print(est.test_conditional_independence('x', 'Z', method='chi_square', tol=0.05))
print(est.test_conditional_independence('x', 'Y', method='chi_square', tol=0.05))
print(est.test_conditional_independence('x', 'Y2', method='chi_square', tol=0.05))
print(est.test_conditional_independence('x', 'Y3', method='chi_square', tol=0.05))
print(est.test_conditional_independence('x', 'Y4', method='chi_square', tol=0.05))
print('=====')
print(est.test_conditional_independence('Z', 'Y', method='chi_square', tol=0.05))
print(est.test_conditional_independence('Z', 'Y2', method='chi_square', tol=0.05))
print(est.test_conditional_independence('Z', 'Y3', method='chi_square', tol=0.05))
print(est.test_conditional_independence('Z', 'Y4', method='chi_square', tol=0.05))
print('=====')
print(est.test_conditional_independence('Y', 'Y2', method='chi_square', tol=0.05))
print(est.test_conditional_independence('Y', 'Y3', method='chi_square', tol=0.05))
print(est.test_conditional_independence('Y', 'Y4', method='chi_square', tol=0.05))
print('=====')
print(est.test_conditional_independence('Y2', 'Y3', method='chi_square', tol=0.05))
print(est.test_conditional_independence('Y2', 'Y4', method='chi_square', tol=0.05))
print('=====')
print(est.test_conditional_independence('Y3', 'Y4', method='chi_square', tol=0.05))
print('=====')

True
False
True
False
False
=====
False
True
False
False
=====
True
False
False
=====
False
False
=====
False
=====


### PCアルゴリズムによる因果探索：1次の独立性

In [69]:
from pgmpy.estimators import ConstraintBasedEstimator

est = ConstraintBasedEstimator(df_bin)

# 0次の独立性の検定
print(est.test_conditional_independence('x', 'Z', ['Y'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('x', 'Z', ['Y3'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('x', 'Z', ['Y4'], method='chi_square', tol=0.05))
print('=====')
print(est.test_conditional_independence('x', 'Y', ['Z'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('x', 'Y', ['Y3'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('x', 'Y', ['Y4'], method='chi_square', tol=0.05))
print('=====')
print(est.test_conditional_independence('x', 'Y3', ['Z'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('x', 'Y3', ['Y'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('x', 'Y3', ['Y4'], method='chi_square', tol=0.05))
print('=====')
print(est.test_conditional_independence('x', 'Y4', ['Z'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('x', 'Y4', ['Y'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('x', 'Y4', ['Y3'], method='chi_square', tol=0.05))
print('=====')

False
False
False
=====
False
False
False
=====
False
True
False
=====
False
True
True
=====


In [70]:
# 1次の独立性の検定 変数z
print(est.test_conditional_independence('Z', 'x', ['Y'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Z', 'x', ['Y3'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Z', 'x', ['Y4'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Z', 'Y', ['x'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Z', 'Y', ['Y3'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Z', 'Y', ['Y4'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Z', 'Y3', ['x'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Z', 'Y3', ['Y'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Z', 'Y3', ['Y4'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Z', 'Y4', ['x'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Z', 'Y4', ['Y'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Z', 'Y4', ['Y3'], method='chi_square', tol=0.05))

False
False
False
False
False
False
False
True
False
False
True
True


In [71]:
# 1次の独立性の検定 変数Y
print(est.test_conditional_independence('Y', 'x', ['Z'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Y', 'x', ['Y3'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Y', 'x', ['Y4'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Y', 'Z', ['x'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Y', 'Z', ['Y3'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Y', 'Z', ['Y4'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Y', 'Y3', ['x'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Y', 'Y3', ['Z'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Y', 'Y3', ['Y4'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Y', 'Y4', ['x'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Y', 'Y4', ['Z'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Y', 'Y4', ['Y3'], method='chi_square', tol=0.05))

False
False
False
False
False
False
False
False
False
False
False
True


In [72]:
# 1次の独立性の検定 変数Y2
print(est.test_conditional_independence('Y2', 'Y3', ['Y4'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Y2', 'Y4', ['Y3'], method='chi_square', tol=0.05))

False
True


In [73]:
# 1次の独立性の検定 変数Y3
print(est.test_conditional_independence('Y3', 'Y', ['Y2'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Y3', 'Y', ['Y4'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Y3', 'Y2', ['Y'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Y3', 'Y2', ['Y4'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Y3', 'Y4', ['Y'], method='chi_square', tol=0.05))
print(est.test_conditional_independence('Y3', 'Y4', ['Y2'], method='chi_square', tol=0.05))

False
False
False
False
False
False


### オリエンテーションルールによる方向付け

In [74]:
# オリエンテーションフェーズでの方向付け
# x-Y-Y3
print(est.test_conditional_independence('x', 'Y3', ['Y'], method='chi_square', tol=0.05))

# Z-Y-Y3
print(est.test_conditional_independence('Z', 'Y3', ['Y'], method='chi_square', tol=0.05))

# Y-Y3-Y2
print(est.test_conditional_independence('Y', 'Y2', ['Y3'], method='chi_square', tol=0.05))

# Y2-Y3-Y4
print(est.test_conditional_independence('Y2', 'Y4', ['Y3'], method='chi_square', tol=0.05))



True
True
False
True


### ベイジアンネットワークでの推論

In [78]:
from pgmpy.models import BayesianModel
from pgmpy.inference import VariableElimination

# モデルを構築
model = BayesianModel([('x', 'Z'), ('x', 'Y'), ('Z', 'Y'), ('Y', 'Y3'), ('Y2', 'Y3'), ('Y3', 'Y4')])

# モデルにデータを与える
model.fit(df_bin)

# 変数消去アルゴリズムで推論
infer = VariableElimination(model)
obserbed_data = {'Z':0,
                   'Y3':3}
x_dist = infer.query(['x'], evidence=obserbed_data)
print(x_dist)
print('=================')
obserbed_data = {'Z':1,
                   'Y3':3}
x_dist = infer.query(['x'], evidence=obserbed_data)
print(x_dist)
print('=================')
obserbed_data = {'Z':1,
                   'Y3':5}
x_dist = infer.query(['x'], evidence=obserbed_data)
print(x_dist)

Eliminating: Y: 100%|██████████| 3/3 [00:00<00:00, 537.94it/s]

+------+----------+
| x    |   phi(x) |
+======+==========+
| x(1) |   0.1515 |
+------+----------+
| x(2) |   0.1599 |
+------+----------+
| x(3) |   0.2154 |
+------+----------+
| x(4) |   0.2191 |
+------+----------+
| x(5) |   0.2541 |
+------+----------+
+------+----------+
| x    |   phi(x) |
+======+==========+
| x(1) |   0.2766 |
+------+----------+
| x(2) |   0.1874 |
+------+----------+
| x(3) |   0.2434 |
+------+----------+
| x(4) |   0.1704 |
+------+----------+
| x(5) |   0.1222 |
+------+----------+
+------+----------+
| x    |   phi(x) |
+======+==========+
| x(1) |   0.0038 |
+------+----------+
| x(2) |   0.0056 |
+------+----------+
| x(3) |   0.0524 |
+------+----------+
| x(4) |   0.2658 |
+------+----------+
| x(5) |   0.6724 |
+------+----------+
